In [ ]:
pip install langchain langchain_openai langchain_community chromadb

In [ ]:
from google.colab import drive
import os

# 먼저 구글 드라이브 마운트
drive.mount('/content/drive')

In [ ]:
import os
from dotenv import load_dotenv

# .env 파일에서 환경 변수 로드
load_dotenv("/content/.env")

# 환경 변수에서 API 키 가져오기
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")

# **초기 세팅**

In [ ]:
from langchain_community.vectorstores import Chroma
from langchain_openai import OpenAIEmbeddings
from langchain_community.document_loaders import TextLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

# 문서 로더 설정
loaders = [TextLoader("문서 경로")]

docs = []
for loader in loaders:
  docs.extend(loader.load())

In [ ]:
# 문서 생성을 위한 텍스트 분할기 정의
recursive_spliiter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)

# 문서 분할
split_docs = recursive_splitter.split_documents(docs)

# OpenAIEmbeddings 인스턴스 생성
embeddings = OpenAIEmbeddings()

# Chroma vectorstore 생성
vectorstore = Chroma.from_document(documents=split_docs, embedding=embeddings)

# Chroma vectorstore 기반 리트리버 생성
retriever = vectorstore.as_retriever()

# **가상 문서 생성 체인**

In [ ]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.runnables import RunnablePassthrough, RunnableLambda
from langchain_text_splitters import CharacterTextSplitter

# 1. 가상 문서 생성 체인
def create_virtual_doc_chain():
  system = "당신은 고도로 숙련된 AI입니다."
  user = """
  주어진 질문 '{query}'에 대해 직접적으로 답변하는 가상의 문서를 생성하세요.
  문서의 크기는 {chunk_size} 글자 언저리여야 합니다.
  """

  prompt = ChatPromptTemplate.from_messages([
      ("system", system),
      ("human", user)
  ])
  llm = ChatOpenAI(model="gpt-4o", temperature=0.2)

  return prompt | llm | StrOutputParser()

In [ ]:
# 2. 문서 검색 체인
def create_retrieval_chain():
  return RunnableLambda(lambda x: retriever.get_relevant_documents(x['virtual_doc']))

# 유틸리티 함수
def format_docs(docs):
  return "\n\n".join(doc.page_content for doc in docs)

In [ ]:
# 3. 최종 응답 생성 체인
def create_final_response_chain():
  final_prompt = ChatPromptTemplate.from_template("""
  다음 정보와 질문을 바탕으로 답변해주세요:
  컨텍스트: {context}

  질문: {question}

  답변:
  """)

  final_llm = ChatOpenAI(model="gpt-4o", temperature=0.2)

  return final_prompt | final_llm

In [ ]:
from langchain_core.runnables import RunnableLambda

def print_input_output(input_data, output_data, step_name):
  print(f"\n--- {step_name} ---")
  print(f"Input: {input_data}")
  print(f"Output: {output_data}")
  print("-" * 50)

In [ ]:
def create_pipeline_with_logging():
  virtual_doc_chain = create_virtual_doc_chain()
  retrieval_chain = create_retrieval_chain()
  final_response_chain = create_final_response_chain()

  # 가상 문서 생성 단계
  def virtual_doc_step(x):
    result = {"virtual_doc": virtual_doc_chain.invoke({
        "query": x["question"],
        "chunk_size": 200
    })}
    print_input_output(x, result, "Virtual Doc Generation")

    return {**xm **result}

  # 문서 검색 단계
  def retrieval_step(x):
    result = {"retrieved_docs": retrieval_chain.invoke(x)}
    print_input_output(x, result, "Document Retrieval")

    return {**x, **result}

  # 컨텍스트 포매팅 단계
  def context_formatting_step(x):
    result = {"context": format_docs(x["retrieved_docs"])}
    print_input_output(x, result, "Context Formatting")

    return {**x, **result}

  # 최종 응답 생성 단계
  def final_response_step(x):
    result = final_response_chain.invoke(x)
    print_input_output(x, result, "Final Response Generation")

    return result

  # 전체 파이프라인 구성
  pipeline = (
      RunnableLambda(virtual_doc_step)
      | RunnableLambda(retrieval_step)
      | RunnableLambda(context+formatting_step)
      | RunnableLambda(final_response_step)
  )

  return pipeline

# 파이프라인 객체 생성
pipeline = create_pipeline_with_logging()